# Feature Engineering

### On `Mobile money` transactions for `Mara Bank`

## Sections in this notebook.


- Introduction
    - Project Overview
    - Objectives
    - Dataset Background

- Data Importation
    - Loading Required Libraries
    - Reading Data Files
    - Initial Data Preview

- Feature Creation
    - Dimensionality Reduction
    - Rolling Averages
    - Relationship Features
    - Fraud Detectors
    - Clustering

- Feature Transformation
    - Encoding
    - Normalization

- Feature Selection
    - Correlation

- Insights and Findings
    - Key Patterns
    - Anomalies
    - Business Insights
    - Recommendations

## Introduction

- Project Overview
- Objectives
- Dataset Background

### Project Overview

Imagine millions of transactions flowing every second through Mara Bank’s mobile money platform from airtime purchases, bill payments, transfers, and more; connecting people from Lagos to remote villages.

Within this constant flow are subtle signals: timing quirks, unusual transaction amounts, sudden changes in behavior. These signals aren’t always obvious, but they can make or break a fraud detection model.

This project focuses on feature engineering, turning raw transaction logs into meaningful variables that a machine can actually learn from.

### Project Objective

The Key objectives are:

- Refine the raw data: Ensure consistency, handle missing values, and normalize fields so features can be reliably created.

- Craft behavioral features: Capture user transaction habits (frequency, average amount, peak transaction hours, device/location patterns).

- Build anomaly indicators: Engineer flags for rare or suspicious events (sudden spikes in amount, rapid-fire transactions, cross-location jumps).

- Time-based intelligence: Generate features from transaction timestamps (day-of-week trends, session bursts, time since last activity).

- Transaction network insights: Create features linking users, merchants, and recipients to reveal unusual transaction paths or dense clusters.

The ultimate goal: design a strong feature set that maximizes the model’s ability to distinguish normal activity from fraud in real time.

### Background of Dataset

The dataset was generated mimicing the different scenerios transactions can occur in Nigeria. It contains transactions of diferrent banks, however we will be focusing on the transactions that belongs to `Mara Bank` for this project.

This dataset contains the following:

- `amount`: The value of the transaction.
- `balance`: The account balance after the transaction.
- `time`: The timestamp of the transaction.
- `holder`: The account number of the transaction's initiator or recipient.
- `kyc`: The kyc level of the account
- `holder_bvn`: The BVN of the transaction's initiator or recipient.
- `holder_bank`: The bank of the related party.
- `related`: The account number or entity related to the transaction (e.g., recipient account, ATM bank).
- `related_bvn`: The BVN of the related party.
- `related_bank`: The bank of the related party.
- `state`, `latitude`, `longitude`: Location details of the transaction.
- `status`: The outcome of the transaction (e.g., 'SUCCESS', 'FAILED').
- `type`: The transaction type (e.g., 'DEBIT', 'CREDIT').
- `category`: The specific class of transaction (e.g., 'OPENING', 'WITHDRAWAL', 'PAYMENT', 'TRANSFER', 'REVERSAL', 'BILL').
- `channel`: The channel used for the transaction (e.g., 'CARD', 'APP', 'USSD').
- `device`: The device used for the transaction (e.g., 'ATM-001', 'MOBILE-003') .
- `nonce`: A unique identifier for related transactions.
- `reported`: Marks reported transactions?
- `kyc`: The kyc leve of the holder
- `merchant`: Is the holder a POS operator
- `hour`: The hour of the transaction
- `week_day`: The day of the week for the transaction
- `month`: The month of the transaction
- `month_day`: The day of the month for the transaction
- `date`: The date of the transaction

## Data Importation

- Loading Required Libraries
- Reading Data Files
- Initial Data Preview

### Loading required libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import modules

import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from src.lib.analytics import tracker, engineer, analyst, detector
from src.lib.store.load_data import load_data

### Reading data files

In [ ]:
# Load the transactions dataset for the project 
df = load_data('analyzed_transactions.csv', parse_dates=['time'])

### Initial data preview

In [ ]:
# Preview the dataset
df.head()

In [ ]:
# Set the format for pandas to display floats
pd.set_option('display.float_format', '{:.2f}'.format)

# Set the theme for our visualization
plt.style.use('dark_background')
sns.set_theme(style="darkgrid")

## Feature Creation

- Dimensionality Reduction
- Rolling Averages
- Relationship Features
- Fraud Detectors
- Clustering

### Dimensionality Reduction

We will reducing the dimensions of any categorical column that requires it. eg. `fraudulence`.

In [ ]:
# Let's name our dataset
df_dimensioned = df.copy()

In [ ]:
# What are the categorical columns
print(df_dimensioned.dtypes.to_frame())

While exploring our dataset, we noticed that most of the transactions are of the category of `withdrawal` and `deposit`.

In [ ]:
# Visualize the distribution of category
df_dimensioned['category'].value_counts().plot(kind='bar')

The difference is too big, and can confuse the machine learning model.

Most of this transactions that have been categorized as `withdrawal` or `deposit` are actually mobile transfers.

Hence we are going to reduce dimension of the `category` feature by renaming these categories as withdrawal.

In [ ]:
# Visualize the distribution of category of transactions made using the APP channel
df_dimensioned[df_dimensioned['channel'] == 'APP']['category'].value_counts().plot(kind='bar')

In [ ]:
# Change mobile withdrawals to transfer
df_dimensioned.loc[(df_dimensioned['channel'] == 'APP') & (df_dimensioned['category'] == 'WITHDRAWAL'), 'category'] = 'TRANSFER'

# Change mobile deposits to transfer
df_dimensioned.loc[(df_dimensioned['channel'] == 'APP') & (df_dimensioned['category'] == 'DEPOSIT'), 'category'] = 'TRANSFER'

In [ ]:
# Visualize the distribution of category
df_dimensioned['category'].value_counts().plot(kind='bar')

### Relating Features

There are alot of relationships between our features like:

- `holder` and `related`. This can be 1 person or a beneficiary.

- `device` and (`holder`/`holder_bvn`), does the device belong to this holder?

- `channel` and (`holder`/`holder_bvn`), how frequent does this user transact using this channel?

- `time` and (`holder`/`holder_bvn`), Is this an unusual time for this user to perform such type of transaction?

- `location` and (`holder`/`holder_bvn`), Is this location too far from the users normal or last known location?

- `amount` and (`holder`/`holder_bvn`), Is this a weird amoumnt for this user.

- `balance` and (`holder`/`holder_bvn`), What is the frequency of this user having this range of balance?

- `reported` and (`holder`/`holder_bvn`), Any reported transaction is a red flag, multiple reported transaction for a user, the user is doing something we should investigate.

- `reversals` and (`holder`/`holder_bvn`), Does the user frequently reverse transactions?

- `device` and `reversals`, is this device prune to reversing transactions?

- `amount`, `balance` and `user`.

In [ ]:
# Name our dataset
df_engr = df_dimensioned.copy()

In [ ]:
# Mark transactions where both the holder_bvn and related_bvn are the same (Same individual)
df_engr['sub_account'] = df_engr['holder_bvn'] == df_engr['related_bvn']

In [ ]:
# How many times did this situation occur?
df_engr['sub_account'].value_counts()

Observation:

Only one user transacted with a `sub_account`.

What is the relationship between the `amount` and `balance`

- Mark transactions with large `amount`; From 100000.
- Mark situations when `balance` got drained or pumped.
- Then determine situations when `amount` is large and `balance` got drained or pumped.

In [ ]:
transaction_limits = {1: 50_000, 2: 200_000, 3: 5_000_000}

In [ ]:
df_engr['large_amount'] = df_engr[['kyc', 'amount']].apply(
    lambda trx: 
    transaction_limits[trx['kyc']] < trx['amount'], 
    axis=1
)

In [ ]:
plt.title('Distribution of large amount')
df_engr['large_amount'].value_counts().plot(kind='bar');

In [ ]:
plt.title('Relationship showing large amounts')
sns.scatterplot(data=df_engr, x='balance', y='amount', hue='large_amount');

In [ ]:
df_engr['balance_jump'] = df_engr.apply(
    lambda row: -row['amount'] if row['type'] == 'DEBIT' else row['amount'],
    axis=1
)

In [ ]:
df_engr['previous_balance'] = df_engr.apply(
    lambda row: row['balance'] - row['balance_jump'],
    axis=1
)

In [ ]:
df_engr['balance_jump_rate'] = df_engr.apply(
    lambda row: row['balance_jump'] / max(row['previous_balance'], 1),
    axis=1
)

In [ ]:
df_engr['balance_jump_rate_absolute'] = df_engr.apply(
    lambda row: abs(row['balance_jump_rate']),
    axis=1
)

In [ ]:
plt.title('Relationship of jump in balance')
sns.scatterplot(data=df_engr, x='previous_balance', y='balance_jump', hue='large_amount');

In [ ]:
df_engr['drained_balance'] = df_engr['balance_jump_rate'] < -.9

In [ ]:
plt.title('Distribution of drained balance')
df_engr['drained_balance'].value_counts().plot(kind='bar');

Wow, no balance got drained.

In [ ]:
plt.title('Relationship of drained balance')
sns.scatterplot(data=df_engr, x='previous_balance', y='balance', hue='drained_balance');

In [ ]:
df_engr['pumped_balance'] = df_engr['balance_jump_rate'] > .9

In [ ]:
plt.title('Distribution of pumped balance')
df_engr['pumped_balance'].value_counts().plot(kind='bar');

In [ ]:
plt.title('Relationship of pumped balance')
sns.scatterplot(data=df_engr, x='previous_balance', y='balance', hue='pumped_balance');

In [ ]:
df_engr['large_amount_drain'] = df_engr['large_amount'] & df_engr['drained_balance']

In [ ]:
plt.title('Distribution of large amount drains')
df_engr['large_amount_drain'].value_counts().plot(kind='bar');

In [ ]:
plt.title('Relationship of drained balance by large amount')
sns.scatterplot(data=df_engr, x='previous_balance', y='balance_jump', hue='large_amount_drain');

In [ ]:
df_engr['large_amount_pump'] = df_engr['large_amount'] & df_engr['pumped_balance']

In [ ]:
plt.title('Distribution of pumped balance by large amount')
df_engr['large_amount_pump'].value_counts().plot(kind='bar')

In [ ]:
plt.title('Relationship of pumped balance by large amount')
sns.scatterplot(data=df_engr, x='previous_balance', y='balance_jump', hue='large_amount_pump');

In [ ]:
df_engr

What is the frequency between the `user` and the following?

`location`

We will determine this with the help of a special function called `distance_from_home` from the `engineer` module.

`distance_from_home` simply `calculates` how many the distance between the `last transaction` and the `current transaction`

In [ ]:
df_engr['distance_from_home (km)'] = tracker.hound(df_engr[['holder', 'latitude', 'longitude', 'holder_latitude', 'holder_longitude', 'time']], lambda d,t: engineer.distance_from_home(d, t, 'holder'))

In [ ]:
df_engr['far_distance'] = df_engr['distance_from_home (km)'] >= 500

In [ ]:
plt.title('Plot of Far distance ratio')
plt.ylabel('Frequency')
plt.xlabel('Far Distance')
df_engr['far_distance'].value_counts().plot(kind='bar');

What is the frequency between the `user` and the following?

`related`, `device`, `channel`, `state`

We will determine this with the help of a special function called `get_count_relations_frequency` from the `engineer` module.

It simply `counts` how many times the `user` has had transactions with `similar` features to the current `transaction`.

In [ ]:
df_relating = df_engr.copy()

In [ ]:
# Get the frequency of features for holder
df_relating = engineer.get_count_relations_frequency(df_relating, 'holder', ['related', 'device', 'channel'])

In [ ]:
# Get the frequency of features for holder_bvn
df_relating = engineer.get_count_relations_frequency(df_relating, 'holder_bvn', ['related_bvn', 'device', 'channel'])

In [ ]:
df_relating['holder_device_count_frequency'].value_counts().plot(kind='bar')

In [ ]:
df_relating['is_opening_device'] = df_relating['device'] == df_relating['opening_device']

What is the frequency between the `holder` and the following?

`hour`, `amount`, `balance`, `balance_jump` and `balance_jump_rate`

We will determine this with the help of a special function called `get_bound_relations_frequency` from the `engineer` module.

It simply `counts` how many times the `user` has had transactions with features `around` the features of the current `transaction`.

In [ ]:
# Get the frequency of features around the current transaction features for holder
df_relating = engineer.get_bounds(df_relating)

What is the frequency between the `user` and the occurance of the following?

`REVERSAL` and `REPORTED`

We will determine this with the help of a special function called `get_occurance` from the `engineer` module.

It simply `calculates` how many the a `unique` kind of situation has occured in the past.


In [ ]:
# Get the number of time the events occured for holder
df_relating = engineer.get_holder_occurance(df_relating)

In [ ]:
# Get the number of time the events occured for holder bvn
df_relating = engineer.get_holder_bvn_occurance(df_relating)

In [ ]:
# Get the number of time the events occured for related
df_relating = engineer.get_related_occurance(df_relating)

In [ ]:
# Get the number of time the events occured for related_bvn
df_relating = engineer.get_related_bvn_occurance(df_relating)

In [ ]:
df_relating

In [ ]:
df_relating.isna().sum().value_counts()

### Rolling Averages

Let's get the rolling averages of `balance`, `amount`, `balance_jump` and `balance_jump_rate` for `days`, `week` and `month`.

In [ ]:
df_rolling = tracker.get_rolling(df_relating)

In [ ]:
df_rolling

In [ ]:
df_rolling.isna().sum().sort_values()

In [ ]:
df_rolling.to_csv(f'../datasets/engineered_transactions.csv', index=False)

## Feature Transformation

In [ ]:
# Naming our dataset
df_transformed = df_rolling.copy()

In [ ]:
# Drop the time feature, we have extracted everything from it.
df_transformed.drop(columns='time', inplace=True)

### Feature Encoding

We will encode our discrete features so our future models will learn well from our dataset.

This will be done with the help of our special function `encoder` from `engineer` module.

It uses `LabelEncoder`.

In [ ]:
# Encode the discrete features
discrete_features = df_transformed.select_dtypes(include=['object']).columns.tolist()
discrete_features

In [ ]:
# Encode discrete features
encoded = engineer.encoder(df_transformed, discrete_features)
df_transformed[discrete_features] = encoded

In [ ]:
# Preview the dataset
df_transformed

### Feature Normalization

We will encode our discrete features so our future models will learn well from our dataset.

This will be done with the help of our special function `encoder` from `engineer` module.

It uses `MinMaxScaler`.

In [ ]:
# Normalize the dataset
scaler = RobustScaler()
scaled_data = scaler.fit_transform(df_transformed)

In [ ]:
# replace the encoded data with the normalized data
df_transformed = pd.DataFrame(scaled_data, columns=df_transformed.columns)

In [ ]:
# Preview the dataframe
df_transformed

## Anomaly Detection

Most transactions seem normal, and to uncover fraudulent pattern are hard.

That is why we will be beconing on a special sklearn estimator `Isolation Forest` that is wrapped inside our custom function `check_anomaly` within the engineer module. 

It will help classify between fraud and non-fraud transcations.

We will also get a score of how fraudulent the transactions are.

In [ ]:
df_unsual = df_transformed.copy()

In [ ]:
df_unsual = detector.unsual_amount(df_unsual)
df_unsual[['unsual_amount_score', 'unsual_amount']].corr()

In [ ]:
df_unsual = detector.unsual_balance(df_unsual)
df_unsual[['unsual_balance_score', 'unsual_balance']].corr()

In [ ]:
df_unsual = detector.unsual_location(df_unsual)
df_unsual[['unsual_location_score', 'unsual_location']].corr()

In [ ]:
df_unsual = detector.unsual_time(df_unsual)
df_unsual[['unsual_time_score', 'unsual_time']].corr()

In [ ]:
df_unsual = detector.unsual_device(df_unsual)
df_unsual[['unsual_device_score', 'unsual_device']].corr()

In [ ]:
# Classify transactions
df_classified = engineer.anomalize(df_unsual, 'fraud')

In [ ]:
df_classified

In [ ]:
# Plot the fraud score
plt.title('Fraud score distribution')
df_classified['fraud_score'].plot(kind='hist')

In [ ]:
# Plot of fraud distribution
plt.title('Fraud distribution')
df_classified['fraud'].value_counts().plot(kind='bar')
plt.xlabel('Fraud')
plt.ylabel('Frequency')

In [ ]:
df_classified['fraud'].value_counts(normalize=True)

In [ ]:
# What is the correlation between fraud and fraud_score
df_classified[['fraud_score', 'fraud']].corr()

In [ ]:
df_classified[df_classified['fraud'] == True].groupby('holder')['fraud'].value_counts().sort_values()

In [ ]:
# Save the dataset
df_classified.to_csv(f'../datasets/classified_transactions.csv', index=False)

## Key Insights and Recommendations

A whole lot more important features can still be engineered from this dataset like:
- Relationships between time and distance
- Did the other party report the transaction
- ...

Also alot more unsual relationships can still be captured, like:
- Relationships between the holder and related, holder_bvn and related_bvn.
- Unsual reversals
- Unsual reporting
- etc.


For now we will continue with the features we already have.
